Topic Mining

In [42]:
from sklearn.datasets import load_files
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

random_state = 0
DATA_DIR = "C:\\Users\\bpanda8\\OneDrive - DXC Production\\Desktop\\Text_Analytics\\Topic Mining\\bbc"
data = load_files(DATA_DIR, encoding="utf-8", decode_error="replace", random_state=random_state)
df = pd.DataFrame(list(zip(data['data'], data['target'])), columns=['text', 'label'])
df.shape
# df.to_csv('bbc_data.csv')

(2225, 2)

In [2]:
def only_nouns(texts):
    output = []
    for doc in nlp.pipe(texts):
        noun_text = " ".join(token.lemma_ for token in doc if token.pos_ == 'NOUN')
        output.append(noun_text)
    return output


df['text'] = only_nouns(df['text'])

df.head()

,text,label
0,boss award executive business magazine title p...,0
1,copy bumper sale fi shooter game copy sale com...,4
2,climate warning climate change control decade ...,2
3,success week race track bronze injury season i...,3
4,tory rethink association candidate election ag...,2


In [3]:
# number of topics to extract
## NMF = Non-Negative Matrix Factorization
n_topics = 5

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = TfidfVectorizer(max_features=5000, stop_words="english", max_df=0.95, min_df=2)
features = vec.fit_transform(df.text)

from sklearn.decomposition import NMF
cls = NMF(n_components=n_topics, random_state=random_state)
cls.fit(features)

NMF(n_components=5, random_state=0)

In [45]:
# list of unique words found by the vectorizer
feature_names = vec.get_feature_names()

# number of most influencing words to display per topic
n_top_words = 10

for i, topic_vec in enumerate(cls.components_):
    print(i, end=' ')
    # topic_vec.argsort() produces a new array
    # in which word_index with the least score is the
    # first array element and word_index with highest
    # score is the last array element. Then using a
    # fancy indexing [-1: -n_top_words-1:-1], we are
    # slicing the array from its end in such a way that
    # top `n_top_words` word_index with highest scores
    # are returned in desceding order
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        print(feature_names[fid], end=' ')
    print()

0 growth sale economy year company share market rate firm price 
1 game player match team injury club time coach win season 
2 film award actor actress star director nomination movie year festival 
3 election party government tax minister leader people campaign chancellor plan 
4 phone people technology music service user software computer broadband network 


In [44]:
new_articles = [
    "The owner of Apple and Microsoft likes rugby"]
    
# first transform the text into features using vec
# then pass it to transform of cls
# the result will be a matrix of shape [2, 10]
# then we sort the topic id based on the score using argsort
# and take the last one (with the highest score) for each row using `[:,-1]` indexing
cls.transform(vec.transform(new_articles)).argsort(axis=1)[:,-1]

array([1], dtype=int64)

LDA
LSA
RNN
BERT